# Applying the Hubbard Hamiltonian to a Vector

$ H = t \sum_{i\sigma} \left(c^{\dagger}_{i\sigma}c_{i+1\sigma}+c^{\dagger}_{i+c\sigma}c_{i\sigma}\right) + U \sum_{i} c^{\dagger}_{i\uparrow}c_{i\uparrow}c^{\dagger}_{i\downarrow}c_{i\downarrow}$

The acction of this Hamiltonian is to move electrons up and down one site and then to count the number of electron pairs.

We can define this action without referenceing a large matrix.  Let us break up the statevectors so that the first $N$ sites refere to spin up and the second $N$ referes to spin down.  For example, for a four site system, $|01010001>$ has two spin up electrons on sites 2 and 4 and one spin down electron on site 4.

## Action of the t term

The idea is as follow for a vector V:

    1) define a mask M which zero everywhere excep site i and i +1
    
    2) perform bitwise AND: K = M & V
    
    3) perform bitwise XOR: L = K ^ M
    
    4) check if L is zero or if L = M
    
        i) if so then this does not contribute
        
        ii) if not then V_new = I - K + L

In [2]:
import numpy as np

In [3]:
V = np.array([0,0,0,1,0,0,1,0])
def update_t(V):
    N = len(V)
    Nu = int(N/2)
    Vn_l = []
    for i in range(0,Nu):
        #Spin up
        M = [0 for i in range(0,N)]
        M[i] = 1
        M[np.mod(i+1,Nu)] = 1
        K = M*V
        L = np.abs( K - M )
        if sum(L) > 0 and sum(np.abs(L - M)) > 0:
            Vn = V - K + L
        else:
            Vn = 0
        Vn_l.append(Vn)
        #Spin down
        M = [0 for i in range(0,N)]
        M[Nu+i] = 1
        M[Nu+np.mod(i+1,Nu)] = 1
        K = M*V
        L = np.abs( K - M )
        if sum(L) > 0 and sum(np.abs(L - M)) > 0:
            Vn = V - K + L
        else:
            Vn = 0
        Vn_l.append(Vn)
    return Vn_l

print(V)
update_t(V)

[0 0 0 1 0 0 1 0]


[0,
 0,
 0,
 array([0, 0, 0, 1, 0, 1, 0, 0]),
 array([0, 0, 1, 0, 0, 0, 1, 0]),
 array([0, 0, 0, 1, 0, 0, 0, 1]),
 array([1, 0, 0, 0, 0, 0, 1, 0]),
 0]

## Action of the U term

For a vector V:
    
    1) define a mask M which zero everywhere excep site i with spin up and spin down
    
    2) perform a bitwise AND: K = M & V
    
    3) check if K = M
        
        i) if so then return V
        
        ii) else return 0

In [4]:
V = np.array([0,1,1,0,0,1,0,1])
def update_u(V):
    N = len(V)
    Nu = int(N/2)
    Vn_l = []
    for i in range(0,Nu):
        #Spin up
        M = [0 for i in range(0,N)]
        M[i] = 1
        M[Nu + i] = 1
        K = M*V
        if np.sum( K - M ) == 0:
            Vn = V
        else:
            Vn = 0
        Vn_l.append(Vn)
    return Vn_l

print(V)
update_u(V)

[0 1 1 0 0 1 0 1]


[0, array([0, 1, 1, 0, 0, 1, 0, 1]), 0, 0]

Here I have saved the vectors as numpy arrays.  For a real implementation the vecotrs should be stored as integers.  

# Working in terms of binary integers

In [133]:
# A function to print out the binary number
def bi(num):
    bi = bin(num)
    out = ""
    for i in range(2,len(bi)):
        out = out + bi[i]
    return out

In [11]:
print(bin(21))
print_bi(21)

0b10101


'10101'

## Action of the t term in binary

The idea is as follow for a vector V:

    1) define a mask M which zero everywhere excep site i and i +1: M = 2^i + 2^{i+1}
    
    2) perform bitwise AND: K = M & V
    
    3) perform bitwise XOR: L = K ^ M
    
    4) check if L is zero or if L = M
    
        i) if so then this does not contribute
        
        ii) if not then V_new = I - K + L

In [128]:
def bi_t(V,N):
    Nu = int(N/2)
    Vn_l = []
    for i in range(0,Nu):
        #Spin up
        M = 2**i + 2**np.mod(i+1,Nu)
        K = M & V
        L = K ^ M
        if L != 0 and L != M:
            Vn = V - K + L
            #print(i,':',bi(Vn))
            Vn_l.append(Vn)
        #Spin down
        M = 2**(Nu+i) + 2**(Nu+np.mod(i+1,Nu))
        K = M & V
        L = K ^ M
        if L != 0 and L != M:
            Vn = V - K + L
            #print(i,':',bi(Vn))
            Vn_l.append(Vn)
    return Vn_l

In [135]:
print(bi(41))
out = bi_t(41,6)
print('......')
for i in range(len(out)):
    print(bi(out[i]))

101001
......
101010
110001
11001
101100


## Action of the U term in binary

For a vector V:
    
    1) define a mask M which zero everywhere excep site i with spin up and spin down: M = 2^i + 2^{i+N/2}
    
    2) perform a bitwise AND: K = M & V
    
    3) check if K = M
        
        i) if so then return V
        
        ii) else return 0

In [136]:
def bi_u(V,N):
    Nu = int(N/2)
    Vn_l = []
    for i in range(0,Nu):
        M = 2**i + 2**(i+Nu)
        K = M & V
        if K == M:
            Vn = V
            Vn_l.append(Vn)
    return Vn_l

In [143]:
V=38
print(bi(V))
out = bi_u(V,6)
print('......')
for i in range(len(out)):
    print(bi(out[i]))

100110
......
100110
